## Raspagem dos resultados das provas da Maratona do Rio 2019 
Meia Maratona, Maratona, Desafio Cidade Maravilhosa, Prova de 5 e 10km e Maratoninha Gloob

In [1]:
import os
import mechanize
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from time import time
from time import sleep
from random import randint

In [3]:
url_base = 'https://globalcronometragem.com.br/evento/'
DATASET_FOLDER  = 'dataset'
MEIA_MARATONA = "meiadorio2019"

br = mechanize.Browser()
br.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.63 Safari/535.7')]

### Meia Maratona

In [4]:
#with open("temp-urls-individuais.txt", "w") as f:
#    for i in url_individuais:
#        f.write(i + "\n")

In [24]:
urls_resultado_individual = []

In [26]:
with open("temp-urls-individuais.txt", "r") as f:
    for linha in f:
        urls_resultado_individual.append(linha[:-1])

In [27]:
len(urls_resultado_individual)

12019

In [127]:
resultado = get_resultado_individual_detalhado("https://globalcronometragem.com.br/evento/maratonadorio2019/?id=781620")

In [128]:
resultado

{'Colocação Geral': '671',
 'Colocação pela faixa etária': '1',
 'Equipe': 'PRETÃ\x89 PIRACICABA-SP ',
 'Modalidade': '42',
 'N. Peito': '5401',
 'Nome do atleta': 'JOSE ANTONIO TEIXEIRA LEITE',
 'Sexo': 'M',
 'Tempo bruto': '04:01:59',
 'Tempo líquido': '03:35:19'}

### Funções Utilitárias

In [5]:
# Recupera a listagem de categorias da prova do evento
def get_categorias(evento):
    
    br.open(url_base + MEIA_MARATONA)
    html = br.response().read()
    
    cat_soup = BeautifulSoup(html, "html.parser")
    select_categorias = cat_soup.find(class_ = "form-control campocategoria")
    
    if select_categorias is None:
        return None
    
    options_categorias = select_categorias.find_all("option")
    
    if options_categorias is None:
        return None
    
    options_categorias = options_categorias[1:]
    
    categorias = []
    
    for o in options_categorias:
        categorias.append(o['value'])
    
    return categorias

In [6]:
#Recupera a URL para o resultado individual de cada atleta do evento por categoria
def get_url_resultados_individuais(lista_categorias, evento):
    
    url_resultados_individuais = []

    for categoria in lista_categorias:

        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,10))
        espera = time() - inicio

        print('--> Request ' + categoria + ' ' + str(espera))
        
        params = {'npeito': '', 'campopesquisa': '', 'categoria': categoria}

        data = urllib.parse.urlencode(params)

        br.open(url_base + evento, data = data)
        resp = br.response().read()
        soup = BeautifulSoup(resp, "html.parser")

        div_resultados = soup.find_all(class_ = "campos")

        if div_resultados is None:
            continue

        for resultado in div_resultados:
            a = resultado.find("a")
            
            if a is None:
                continue;
                
            url_individual = a['href']
            
            url_resultados_individuais.append(url_individual)

            print(resultado.find("b").text)
        
    return url_resultados_individuais

In [126]:
#Recupera o resultado individual do atleta
def get_resultado_individual_detalhado(url):

    br.open(url)
    html = br.response().read()

    soup = BeautifulSoup(html, "html.parser")

    div_externa = soup.find_all(class_ = "cimaevento span12")
    
    if div_externa is None:
        return
    
    if len(div_externa) < 2:
        return
    
    div = div_externa[1]

    temp = div.text.split('\n')

    temp = list(filter(lambda x: x != '', temp))[1:-1]

    temp_dic = dict(zip(temp[::2], temp[1::2]))
    
    return temp_dic